In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [3]:
satisfied = pd.read_excel("./input/2020/근로_환경_만족도_2020.xlsx")
satisfied = satisfied.drop(columns="특성별(1)", axis=1)
satisfied = satisfied.set_index("특성별(2)")

satisfied = satisfied.iloc[10:27]


satisfied = satisfied.rename(columns={"2020.3": "불만족", "2020.4": "매우 불만족"})
total_list = []
for i in range(len(satisfied)):
    total = int(satisfied.iloc[i][0]*satisfied.iloc[i][3]/100+0.5)
    total_list.append(total)
# 311 49
total_list[5] += total_list[7]
del total_list[7]
print(total_list)
region = satisfied.index.to_list()
region[5]="대전(세종)"
del region[7]
region
satisfied

[1316, 484, 455, 473, 223, 360, 203, 2020, 223, 316, 331, 285, 251, 436, 530, 127]


c:\Users\dawoo\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\dawoo\AppData\Local\Temp\ipykernel_5616\2162373195.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  total = int(satisfied.iloc[i][0]*satisfied.iloc[i][3]/100+0.5)


,2020,2020.1,2020.2,불만족,매우 불만족,2020.5,2020.6
특성별(2),,,,,,,
서울,9537,6.1,77.8,13.8,1.1,1.2,2.9
부산,3267,10.8,73.4,14.8,0.6,0.3,2.9
대구,2498,5.4,74.8,18.2,1.1,0.6,2.8
인천,2917,6.2,74.3,16.2,2.2,1.1,2.9
광주,1391,10.3,72.3,16,1.1,0.3,2.9
대전,1545,8.4,70.3,20.1,1.2,0.1,2.9
울산,1287,12.1,70.5,15.8,1,0.7,2.9
세종,359,11.7,73.4,13.6,1.3,0,3
경기,13117,7.7,74.9,15.4,1.7,0.4,2.9


In [4]:
local_people = pd.read_excel("./output/지역별_퇴사자3.xlsx")
local_people = local_people.iloc[:, [0, -9, -8, -7]]
local_people = local_people.rename(
    columns={"2020.1": "경력", "2020.2": "입사경험", "2020.3": "신입"})
local_people = local_people.drop(index=0)
list = local_people.iloc[1:, 3]
new = []
for i in range(len(list)):
    num_str = list.iloc[i].replace(",", "")
    # print(num_str)
    new.append(int(num_str))
new

[7500,
 1851,
 1666,
 3525,
 205,
 1962,
 1894,
 11975,
 501,
 3179,
 3434,
 1016,
 650,
 715,
 8486,
 74]

In [5]:
people = pd.read_excel("./input/2020/행정구역_시군구_별_주민등록세대수_2020.xlsx")

list_people = []

for i in range(1, len(people)):
    value = people.iloc[i, 1]
    list_people.append(value)

list_people[5] += list_people[7]
del list_people[7]
pay = pd.read_excel("./input/2020/지역별_평균임금_2020.xlsx")
pay_list = []

for i in range(2,len(pay)):
    value=int(pay.iloc[i,3]*1000000)
    pay_list.append(value)


In [6]:
data = {"지역": region,
        "퇴사자수": new,
        "인구": list_people,
        "평균임금": pay_list,
        "불만족": total_list}


df = pd.DataFrame(data)

mean = int(pd.Series(new).mean()+0.5)
mean

df["퇴사율_높음"] = (df["퇴사자수"] > mean).astype(int)

X = df[["인구", "평균임금"]]
y = df["퇴사율_높음"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("정확도:", accuracy)
df

정확도: 0.75


,지역,퇴사자수,인구,평균임금,불만족,퇴사율_높음
0,서울,7500,4417954,43800000,1316,1
1,부산,1851,1530431,35200000,484,0
2,대구,1666,1056627,35000000,455,0
3,인천,3525,1267956,34100000,473,1
4,광주,205,633582,35900000,223,0
5,대전(세종),1962,797058,38900000,360,0
6,울산,1894,476893,43300000,203,0
7,경기,11975,5676401,38900000,2020,1
8,강원,501,736301,34400000,223,0
9,충북,3179,745644,35800000,316,1


In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [8]:
# import matplotlib.font_manager as fm
# font_path = "C:/Windows/Fonts/malgun.ttf"
# font_name = fm.FontProperties(fname=font_path).get_name()
# sns.boxplot(x='퇴사율_높음', y='퇴사자수', data=df)
# plt.show()

In [9]:
t_stat, p_value = stats.ttest_ind(df[df['퇴사율_높음'] == 1]['퇴사자수'],
                                df[df['퇴사율_높음'] == 0]['퇴사자수'])
print('t-statistic:', t_stat)
print('p-value:', p_value)

t-statistic: 4.62429076523574
p-value: 0.0003938463454160304


In [10]:
X = df["불만족"]
y = df["퇴사율_높음"]

correllation = X.corr(y)
print(correllation)

0.5405239894153957
